# Importing Library


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from tensorflow.keras.utils import to_categorical
from numpy import mean
from numpy import std
from numpy import dstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D

# Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd'/content/drive/MyDrive/Colab Notebooks/Dataset/Softcom Assignment 1'

/content/drive/MyDrive/Colab Notebooks/Dataset/Softcom Assignment 1


In [ ]:
load = pd.read_csv('Preprocessed_Exercise_Deep_Learning.csv');

In [ ]:
df = load.copy()

In [ ]:
df.head()

,Unnamed: 0,user_name,raw_timestamp_part_1,raw_timestamp_part_2,cvtd_timestamp,new_window,num_window,roll_belt,pitch_belt,yaw_belt,...,gyros_dumbbell_x,gyros_dumbbell_y,gyros_dumbbell_z,accel_dumbbell_x,accel_dumbbell_y,accel_dumbbell_z,magnet_dumbbell_x,magnet_dumbbell_y,magnet_dumbbell_z,classe
0,0,eurico,1.322490e+09,34670.0,28/11/2011 14:15,no,1.0,3.70,41.6,-82.8,...,-0.31,0.16,0.08,5.0,21.0,37.0,-471.0,191.0,277.0,E
1,1,eurico,1.322490e+09,62641.0,28/11/2011 14:15,no,1.0,3.66,42.8,-82.5,...,-0.31,0.14,0.07,4.0,22.0,35.0,-472.0,184.0,281.0,E
2,2,eurico,1.322490e+09,70653.0,28/11/2011 14:15,no,1.0,3.58,43.7,-82.3,...,-0.31,0.16,0.05,3.0,23.0,37.0,-468.0,190.0,275.0,E
3,3,eurico,1.322490e+09,82654.0,28/11/2011 14:15,no,1.0,3.56,44.4,-82.1,...,-0.31,0.16,0.07,5.0,24.0,38.0,-469.0,184.0,285.0,E
4,4,eurico,1.322490e+09,90637.0,28/11/2011 14:15,no,1.0,3.57,45.1,-81.9,...,-0.31,0.14,0.07,5.0,23.0,37.0,-468.0,189.0,292.0,E


# Converting dataframe to numpy arrays

In [ ]:
df2 = pd.DataFrame(data = np.array(df['new_window']), columns=['new_window'] )
df2['roll_belt'], df2['pitch_belt'], df2['yaw_belt'] = np.array(df['roll_belt']),  np.array(df['pitch_belt']), np.array(df['yaw_belt'])
df2['roll_dumbbell'], df2['pitch_dumbbell'], df2['yaw_dumbbell']  = np.array(df['roll_dumbbell']),  np.array(df['pitch_dumbbell']), np.array(df['yaw_dumbbell'])
df2['total_accel_belt'], df2['total_accel_dumbbell']  = np.array(df['total_accel_belt']),  np.array(df['total_accel_dumbbell'])
df2['gyros_belt_x'], df2['gyros_belt_y'], df2['gyros_belt_z']  = np.array(df['gyros_belt_x']),  np.array(df['gyros_belt_y']), np.array(df['gyros_belt_z'])
df2['accel_belt_x'], df2['accel_belt_y'], df2['accel_belt_z']  = np.array(df['accel_belt_x']),  np.array(df['accel_belt_y']), np.array(df['accel_belt_z'])
df2['magnet_belt_x'], df2['magnet_belt_y'], df2['magnet_belt_z']  = np.array(df['magnet_belt_x']),  np.array(df['magnet_belt_y']), np.array(df['magnet_belt_z'])
df2['gyros_dumbbell_x'], df2['gyros_dumbbell_y'], df2['gyros_dumbbell_z']  = np.array(df['gyros_dumbbell_x']),  np.array(df['gyros_dumbbell_y']), np.array(df['gyros_dumbbell_z'])
df2['accel_dumbbell_x'], df2['accel_dumbbell_y'], df2['accel_dumbbell_z']  = np.array(df['accel_dumbbell_x']),  np.array(df['accel_dumbbell_y']), np.array(df['accel_dumbbell_z'])
df2['magnet_dumbbell_x'], df2['magnet_dumbbell_y'], df2['magnet_dumbbell_z']  = np.array(df['magnet_dumbbell_x']),  np.array(df['magnet_dumbbell_y']), np.array(df['magnet_dumbbell_z'])
df2['num_window'] = np.array(df['num_window'])
df2['classe'] =  np.array(df['classe'])

In [ ]:
df2.columns

Index(['new_window', 'roll_belt', 'pitch_belt', 'yaw_belt', 'roll_dumbbell',
       'pitch_dumbbell', 'yaw_dumbbell', 'total_accel_belt',
       'total_accel_dumbbell', 'gyros_belt_x', 'gyros_belt_y', 'gyros_belt_z',
       'accel_belt_x', 'accel_belt_y', 'accel_belt_z', 'magnet_belt_x',
       'magnet_belt_y', 'magnet_belt_z', 'gyros_dumbbell_x',
       'gyros_dumbbell_y', 'gyros_dumbbell_z', 'accel_dumbbell_x',
       'accel_dumbbell_y', 'accel_dumbbell_z', 'magnet_dumbbell_x',
       'magnet_dumbbell_y', 'magnet_dumbbell_z', 'num_window', 'classe'],
      dtype='object')

# Creating data of fixed window size

In [ ]:
total = 0
count = 0
dropping_threshold = 27
for i in df2['num_window'].unique():
    val = len( df2[ df2['num_window'] == i ])
    total += val
    if val <=dropping_threshold:
        count += 1
print('Total : ', df2['num_window'].nunique())
print('Average : ', total/df2['num_window'].nunique())
print('Dropped : ', count)

Total :  855
Average :  45.88187134502924
Dropped :  39


In [ ]:
# lst = []
# for i in df2['num_window'].unique():  
#     lst.append(len( df2[ df2['num_window'] == i ]))
# max(lst)

In [ ]:
#Dropping Code
print(len(df2))
for i in df2['num_window'].unique():    
    temp = df2[df2['num_window'] == i]
    if(len(temp)<dropping_threshold):
        df2.drop(df2[df2['num_window'] == i].index, inplace=True)
        # print(len(temp), i ) 
print(len(df2))

39229
38525


In [ ]:
def impute_classe(x):
  if x == 'A':
    return 5
  elif x =='B':
    return 4
  elif x=='C':
    return 3
  elif x=='D':
    return 2
  elif x=='E':
    return 1

In [ ]:
df2['classe'] = df2['classe'].apply(impute_classe)

In [ ]:
column_names = df2.drop(['classe', 'new_window', 'num_window'], axis=1).columns

In [ ]:
global v_stacked
global v_stacked_2
window_size = dropping_threshold
counter = 0
v_stacked_2 = np.zeros((1,1))
target_label = []
for i in df2['num_window'].unique():
    temp = df2[df2['num_window'] == i]
    label = df2[df2['num_window'] == i]['classe'].iloc[0]
    n = int(len(temp)/window_size)
    for j in range(0,n):
        v_stacked = np.zeros(shape=(1, 1))
        for k in column_names:
            temp2 = df2[df2['num_window'] == i][k].to_numpy()
            temp2 = temp2[j*window_size:(j*window_size) + window_size]
            temp2 = temp2.reshape(-1,1)          
            if v_stacked.shape[0] == 1 and v_stacked.shape[1] == 1:
                v_stacked = temp2
            else:
                v_stacked = np.dstack((v_stacked, temp2))   
        target_label.append(label)        
        if v_stacked_2.shape[0] == 1 and v_stacked_2.shape[1] == 1:
            v_stacked_2 = v_stacked.reshape(1,window_size, len(column_names))
        else:
            v_stacked_2 = np.vstack((v_stacked_2, v_stacked.reshape(1,window_size,len(column_names))))

In [ ]:
print(v_stacked_2.shape)

(842, 30, 26)


In [ ]:
print(v_stacked_2[0].shape)

(30, 26)


In [ ]:
print(len(target_label))

842


In [ ]:
target_label = to_categorical(target_label)

In [ ]:
trainX = v_stacked_2[0:800][:][:]
print(trainX.shape)
testX = v_stacked_2[800:-1][:][:]
print(testX.shape)
trainy = target_label[0:800][:][:]
print(trainy.shape)
testy = target_label[800:-1][:][:]
print(testy.shape)

(800, 30, 26)
(41, 30, 26)
(800, 6)
(41, 6)


In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 80, 8
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(Conv1D(filters=6, kernel_size=3, activation='relu', padding='same',input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=9, kernel_size=3, activation='relu', padding='same'))
	model.add(Dropout(0.25))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Conv1D(filters=12, kernel_size=3, activation='relu', padding='same'))
	model.add(Dropout(0.15))
	model.add(LSTM(units=12, activation='relu') 
	model.add(Flatten())
	model.add(Dense(400, activation='relu'))
	model.add(Dropout(0.10))
	model.add(Dense(700, activation='relu'))
	model.add(Dropout(0.15))
	model.add(Dense(1000, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [ ]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
def run_experiment(repeats=10):
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [ ]:
run_experiment()

>#1: 60.976
>#2: 53.659
>#3: 58.537
>#4: 43.902
>#5: 43.902
>#6: 53.659
>#7: 70.732
>#8: 58.537
>#9: 60.976
>#10: 78.049
[60.975611209869385, 53.658539056777954, 58.53658318519592, 43.90243887901306, 43.90243887901306, 53.658539056777954, 70.7317054271698, 58.53658318519592, 60.975611209869385, 78.04877758026123]
Accuracy: 58.293% (+/-10.083)
